<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/FineTuning_VertexAISDK_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installations

In [ ]:
!pip install google-cloud-aiplatform -q
!pip install google-cloud-storage -q
!pip install google-cloud-bigquery -q
!pip install google-cloud-bigquery-storage -q
!pip install google-cloud-aiplatform -q
!pip install datasets -q
!pip install colab-env -q

# Install necessary libraries
!pip install  -q gcsfs==2024.3.1
!pip install  -q accelerate==0.31.0
!pip install  -q transformers==4.45.2
!pip install  -q  datasets==2.19.2
!pip install google-cloud-aiplatform[all] -q
!pip install vertexai  -q
!pip install tensorflow_datasets -q

!pip install google-cloud-aiplatform -q -U

print('\n')


import google.cloud.bigquery
import google.cloud.bigquery_storage
import google.cloud.aiplatform
import google.cloud.storage

print(f"google-cloud-aiplatform: {google.cloud.aiplatform.__version__}")
print(f"google-cloud-storage: {google.cloud.storage.__version__}")
print(f"google-cloud-bigquery: {google.cloud.bigquery.__version__}")
print(f"google-cloud-bigquery-storage: {google.cloud.bigquery_storage.__version__}")
print(f"google-cloud-aiplatform: {google.cloud.aiplatform.__version__}")

In [5]:
import datasets
import colab_env
print('\n\n')
print(f"datasets: {datasets.__version__}")
print(f"colab-env: {colab_env.__version__}")
import google.cloud.aiplatform
import google.cloud.storage
print(f"google-cloud-aiplatform: {google.cloud.aiplatform.__version__}")
print(f"google-cloud-storage: {google.cloud.storage.__version__}")

Mounted at /content/gdrive



datasets: 2.19.2
colab-env: 0.2.0
google-cloud-aiplatform: 1.87.0
google-cloud-storage: 2.19.0


## data preparation

In [ ]:
!gsutil cp gs://{BUCKET_NAME}/cmapss_FD004_train_text.jsonl .
!gsutil cp gs://{BUCKET_NAME}/cmapss_FD004_test_text.jsonl .

In [51]:
import json
import logging

def validate_jsonl_format(file_path, check_transformed_cmapss=True):
    """
    Validates JSONL format and optionally checks for the transformed CMAPSS
    data structure.

    Args:
        file_path (str): Path to the JSONL file.
        check_transformed_cmapss (bool): If True, checks for "prompt" and
                                         "completion" both being strings.
    Returns:
        bool: True if valid, False otherwise.
    """
    with open(file_path, 'r') as f:
        for line in f:
            try:
                data = json.loads(line)
                if "prompt" not in data or "completion" not in data:
                    logging.error(
                        f"Invalid format: Missing 'prompt' or 'completion' in line: {line.strip()}"
                    )
                    return False

                if check_transformed_cmapss:
                    if not isinstance(data["prompt"], str) or not isinstance(
                        data["completion"], str
                    ):
                        logging.error(
                            f"Invalid transformed CMAPSS format in line: {line.strip()}"
                        )
                        return False
            except json.JSONDecodeError:
                logging.error(f"Invalid JSON in line: {line.strip()}")
                return False
    logging.info(f"File '{file_path}' has valid format.")
    return True

# Create a dummy JSONL file for transformed CMAPSS data
transformed_cmapss_file_path = "cmapss_FD004_test_text_transformed.jsonl"
with open(transformed_cmapss_file_path, "w") as f:
    f.write(
        '{"prompt": "Sensor readings from test: 25.1, 1200.5, 45.6", "completion": "185"}\n'
    )
    f.write(
        '{"prompt": "More test data: 25.2, 1199.8, 45.5", "completion": "170"}\n'
    )

# Validate the transformed CMAPSS file
if validate_jsonl_format(
    transformed_cmapss_file_path, check_transformed_cmapss=True
):
    print(f"'{transformed_cmapss_file_path}' has a valid transformed CMAPSS format.")
else:
    print(f"'{transformed_cmapss_file_path}' has an invalid format.")

'cmapss_FD004_test_text_transformed.jsonl' has a valid transformed CMAPSS format.


In [50]:
import json
import logging

def validate_jsonl_format(file_path, check_time_series=True):
    """
    Validates JSONL format and optionally checks for time-series data structure.

    Args:
        file_path (str): Path to the JSONL file.
        check_time_series (bool): If True, checks for "prompt" containing a list
                                   and "completion" being a number.
    Returns:
        bool: True if valid, False otherwise.
    """
    with open(file_path, 'r') as f:
        for line in f:
            try:
                data = json.loads(line)
                if "prompt" not in data or "completion" not in data:
                    logging.error(f"Invalid format: Missing 'prompt' or 'completion' in line: {line.strip()}")
                    return False

                if check_time_series:
                    if not isinstance(data["prompt"], list) or not isinstance(data["completion"], (int, float)):
                        logging.error(f"Invalid time-series format in line: {line.strip()}")
                        return False
            except json.JSONDecodeError:
                logging.error(f"Invalid JSON in line: {line.strip()}")
                return False
    logging.info(f"File '{file_path}' has valid format.")
    return True

In [49]:
import json

def validate_jsonl_format(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            try:
                data = json.loads(line)
                if "prompt" not in data or "completion" not in data:
                    print(f"Invalid format in line: {line}")
                    return False  # Indicate invalid format
            except json.JSONDecodeError:
                print(f"Invalid JSON in line: {line}")
                return False  # Indicate invalid JSON
    return True  # Indicate valid format

# Example usage
if validate_jsonl_format("cmapss_FD004_train_text.jsonl"):
    print("Training data has valid format.")

else:
    print("Training data has invalid format.")

if validate_jsonl_format("cmapss_FD004_test_text.jsonl"):
    print("Testing data has valid format.")
else:
    print("Testing data has invalid format.")

Invalid format in line: {"contents": [{"role": "user", "parts": [{"text": "Engine sensor readings over time: [1.0, 41.9993, 0.8409, 100.0, 445.0, 548.68, 1343.85, 1111.03, 3.91, 5.69, 137.26, 2211.96, 8296.96, 1.01, 41.69, 129.46, 2387.97, 8068.65, 9.3383, 0.02, 328.0, 2212.0, 100.0, 10.48, 6.3841, 2.0, 42.0073, 0.84, 100.0, 445.0, 548.5, 1343.11, 1108.02, 3.91, 5.69, 138.4, 2211.93, 8304.82, 1.01, 41.56, 129.91, 2387.98, 8075.72, 9.3616, 0.02, 330.0, 2212.0, 100.0, 10.58, 6.3506, 3.0, 20.0058, 0.7, 100.0, 491.19, 607.22, 1478.88, 1244.42, 9.35, 13.6, 332.86, 2323.66, 8709.26, 1.07, 43.99, 312.69, 2387.85, 8053.84, 9.1826, 0.02, 363.0, 2324.0, 100.0, 24.59, 14.688, 4.0, 10.0039, 0.25, 100.0, 489.05, 604.1, 1488.19, 1297.5, 10.52, 15.46, 393.88, 2318.63, 8773.14, 1.26, 44.95, 370.86, 2387.83, 8124.24, 8.6464, 0.03, 367.0, 2319.0, 100.0, 28.8, 17.0438, 5.0, 9.999, 0.25, 100.0, 489.05, 604.46, 1490.03, 1294.91, 10.52, 15.46, 393.25, 2318.65, 8766.53, 1.25, 44.9, 370.42, 2387.86, 8123.0, 8

In [ ]:
import json

def transform_jsonl_to_prompt_completion(input_file_path, output_file_path):
    """Transforms chat-style JSONL to prompt-completion JSONL."""
    with open(input_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
        for line in infile:
            try:
                data = json.loads(line)
                # Extract prompt and completion from 'contents'
                prompt = "".join([part["text"] for part in data["contents"][0]["parts"]])  # Assumes user role is first
                completion = str(data.get("completion", "")) # Handle if completion is missing

                # Construct prompt-completion dictionary
                prompt_completion_data = {"prompt": prompt, "completion": completion}

                # Write to output file
                outfile.write(json.dumps(prompt_completion_data) + "\n")

            except (json.JSONDecodeError, KeyError, IndexError) as e:
                print(f"Skipping invalid or unprocessable line: {line.strip()}, Error: {e}")

# Example usage:
input_file_path = "cmapss_FD004_train_text.jsonl"
output_file_path = "cmapss_FD004_train_text_transformed.jsonl"

transform_jsonl_to_prompt_completion(input_file_path, output_file_path)
print(f"Transformed data written to: {output_file_path}")

input_file_path = "cmapss_FD004_test_text.jsonl"
output_file_path = "cmapss_FD004_test_text_transformed.jsonl"

transform_jsonl_to_prompt_completion(input_file_path, output_file_path)
print(f"Transformed data written to: {output_file_path}")


In [ ]:
import json

def validate_jsonl_format(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            try:
                data = json.loads(line)
                if "prompt" not in data or "completion" not in data:
                    print(f"Invalid format in line: {line}")
                    return False  # Indicate invalid format
            except json.JSONDecodeError:
                print(f"Invalid JSON in line: {line}")
                return False  # Indicate invalid JSON
    return True  # Indicate valid format

# Example usage
if validate_jsonl_format("cmapss_FD004_train_text_transformed.jsonl"):
    print("Training data has valid format.")

else:
    print("Training data has invalid format.")

if validate_jsonl_format("cmapss_FD004_test_text_transformed.jsonl"):
    print("Testing data has valid format.")
else:
    print("Testing data has invalid format.")

In [ ]:
!gsutil cp cmapss_FD004_train_text_transformed.jsonl gs://{BUCKET_NAME}/
!gsutil cp cmapss_FD004_test_text_transformed.jsonl gs://{BUCKET_NAME}/

## FINE TUNING - NASA DATASET

In [120]:
!gsutil ls -lh gs://{BUCKET_NAME}/*text*

  1.41 MiB  2025-04-03T05:12:19Z  gs://poc-my-new-staging-bucket-2025-1/cmapss_FD004_test_text.jsonl
  1.39 MiB  2025-04-04T07:15:31Z  gs://poc-my-new-staging-bucket-2025-1/cmapss_FD004_test_text_transformed.jsonl
  1.26 MiB  2025-04-03T05:12:17Z  gs://poc-my-new-staging-bucket-2025-1/cmapss_FD004_train_text.jsonl
  1.24 MiB  2025-04-04T07:15:28Z  gs://poc-my-new-staging-bucket-2025-1/cmapss_FD004_train_text_transformed.jsonl
TOTAL: 4 objects, 5567928 bytes (5.31 MiB)


In [ ]:
!pip install google-cloud-aiplatform -U -q

from google.cloud import aiplatform
import os

# Get project details from environment variables
PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
REGION = os.environ.get("GOOGLE_CLOUD_REGION")


# Initialize Vertex AI with error handling
try:
    aiplatform.init(project=PROJECT_ID, location=REGION)
except Exception as e:
    print(f"Error initializing Vertex AI: {e}")
    exit(1)  # Exit with error code

# List the models with error handling (without filters)
try:
    models = aiplatform.Model.list()  # No filter applied

    print("Available models in Vertex AI:")
    for model in models:
        print(f"- {model.display_name} ({model.resource_name})")
except Exception as e:
    print(f"Error listing models: {e}")
    exit(1)  # Exit with error code

In [121]:
vertexai_index=10
models = aiplatform.Model.list()

name=models[vertexai_index].display_name
resource_name=models[vertexai_index].resource_name
print(f"name: {name}")
print(f"resource_name: {resource_name}")

name: cmapss-text-tuned-gemini-2.0-flash-001-1.0
resource_name: projects/677155171887/locations/us-central1/models/3735556670508498944


In [ ]:
tuned_model_display_name_or_index=9
try:
    # Try loading by display name first
    models = aiplatform.Model.list()
    if models:
        base_model = models[vertexai_index]
        logging.info(f"Loaded {base_model.resource_name} by display name.")
        embedding_dim = base_model.predict(instances=[{"content": "test"}]).predictions[0].shape[0]

        print(f"{base_model.resource_name}")
    else:
        # If not found, assume it's an index
        vertexai_index = int(tuned_model_display_name_or_index)
        models = aiplatform.Model.list()
        if 0 <= vertexai_index < len(models):
            base_model = models[vertexai_index]
            logging.info(f"Loaded {base_model.resource_name} by index.")
        else:
            raise ValueError(f"Invalid vertexai_index or display_name: {tuned_model_display_name_or_index}")
except ValueError as e:
    logging.error(f"Error loading model: {e}")
    raise
except Exception as e:
    logging.error(f"Unexpected error loading model: {e}")
    raise


In [106]:
train_py_content = """
import argparse
import os
import json
import logging
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import time
import subprocess
from google.cloud import storage
import pandas as pd
from typing import List
import numpy as np
import torch.utils.data as data
import vertexai
from google.cloud import aiplatform  # Import aiplatform
import re  # Import the regular expression module
import tensorflow as tf


logging.basicConfig(level=logging.INFO)

# --- Helper Functions ---
def create_gcs_dir(model_dir, bucket_name):  # Add bucket_name as argument
    # Creates a directory in Google Cloud Storage (GCS) if it doesn't exist.
    try:
        storage_client = storage.Client()
        # bucket_name = model_dir.split('/')[2]  <-- Remove this line
        blob_prefix = '/'.join(model_dir.split('/')[3:])
        bucket = storage_client.bucket(bucket_name)

        subdirs = blob_prefix.split('/')
        current_prefix = ''
        for subdir in subdirs:
            current_prefix = os.path.join(current_prefix, subdir)
            blob = bucket.blob(current_prefix + '/')
            blob.upload_from_string('')
            logging.info("Created GCS directory: %s", current_prefix)
    except Exception as e:
        logging.error(f"Error creating GCS directory: {e}")
        raise


def load_jsonl_dataset(data_path, sequence_length=30):
    # Loads a JSONL dataset with the specific format.
    data = []
    try:
        with tf.io.gfile.GFile(data_path, 'r') as f:
            for line in f:
                try:
                    record = json.loads(line)

                    # Extract prompt (sensor readings)
                    prompt_text = record["contents"][0]["parts"][0]["text"]  # Assumes user role is first
                    sensor_readings_match = re.search(r"\[(.*?)\]", prompt_text)
                    if sensor_readings_match:
                        sensor_readings_str = sensor_readings_match.group(1)
                        prompt = [float(x) for x in sensor_readings_str.split(",")]
                    else:
                        logging.warning(f"Could not find sensor readings in text: {prompt_text}")
                        continue  # Skip this line if sensor readings not found

                    # Extract completion (RUL)
                    completion_text = record["contents"][1]["parts"][0]["text"]  # Assumes model role is second
                    completion_match = re.search(r"Remaining Useful Life: (.*)", completion_text)
                    if completion_match:
                        completion = float(completion_match.group(1))
                    else:
                        logging.warning(f"Could not find Remaining Useful Life in text: {completion_text}")
                        continue  # Skip this line if RUL not found

                    data.append((prompt, completion))  # Append the extracted data

                except (json.JSONDecodeError, KeyError, IndexError, ValueError) as e:
                    logging.warning("Skipping invalid JSON line: %r, Error: %s", repr(line), e)

        return data  # Return the extracted data

    except Exception as e:
        logging.error(f"Error loading dataset: {e}")
        raise


# --- Dataset Class ---
class CMAPSSJSONLDataset(data.Dataset):
    def __init__(self, data_path, sequence_length=30, use_gcs=False, model=None, bucket_name=None):
        self.data = []
        self.sequence_length = sequence_length
        self.model = model
        self.bucket_name = bucket_name

        # GCS file handling (if use_gcs is True and data_path is a GCS URI)
        if use_gcs and data_path.startswith('gs://'):
            storage_client = storage.Client()
            # bucket_name = os.environ.get("GOOGLE_CLOUD_BUCKET_NAME")  # Removed redundant line
            blob_name = data_path.replace(f"gs://{self.bucket_name}/", "")  # Use self.bucket_name
            bucket = storage_client.bucket(self.bucket_name)  # Use self.bucket_name
            blob = bucket.blob(blob_name)
            tmp_file = "/tmp/temp_data.jsonl"  # Temporary file for download
            blob.download_to_filename(tmp_file)
            data_file = tmp_file
        else:
            # Local file handling
            data_file = data_path

        with open(data_file, 'r') as f:
            for line in f:
                try:
                    record = json.loads(line)

                    # Extract sensor readings from the first "parts" element
                    text_content = record["contents"][0]["parts"][0]["text"]
                    sensor_readings_match = re.search(r"\[(.*?)\]", text_content)
                    if sensor_readings_match:
                        sensor_readings_str = sensor_readings_match.group(1)
                        prompt = [float(x) for x in sensor_readings_str.split(",")]
                    else:
                        logging.warning(f"Could not find sensor readings in text: {text_content}")
                        continue

                    # Extract completion (Remaining Useful Life) from the second "parts" element
                    completion_text = record["contents"][1]["parts"][0]["text"]
                    completion_match = re.search(r"Remaining Useful Life: (.*)", completion_text)
                    if completion_match:
                        completion = float(completion_match.group(1))
                    else:
                        logging.warning(f"Could not find Remaining Useful Life in text: {completion_text}")
                        continue

                    self.data.append((prompt, completion))

                except (json.JSONDecodeError, KeyError, ValueError, IndexError) as e:
                    logging.warning(f"Skipping invalid line: {line.strip()}, Error: {e}")

        # Remove the temporary file if it was created
        if use_gcs and data_path.startswith('gs://'):
            os.remove(tmp_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    def generate_embeddings(self, model):
        embeddings_data = []
        for prompt, completion in self.data:
            try:
                # Check if prompt is already an embedding (during inference)
                if isinstance(prompt, torch.Tensor):
                    embeddings_tensor = prompt
                else:
                    # Generate embeddings if prompt is a list of sensor readings
                    prompt_text = ','.join(map(str, prompt))  # Convert prompt to string for embedding
                    embeddings = model.predict(instances=[{"content": prompt_text}]).predictions[0]
                    embeddings_tensor = torch.tensor(embeddings, dtype=torch.float32)

                rul = torch.tensor([completion], dtype=torch.float32)  # Assuming 'completion' represents RUL
                embeddings_data.append((embeddings_tensor, rul))
            except Exception as e:
                logging.warning(f"Error generating embeddings for prompt: {prompt}, Error: {e}")
        self.data = embeddings_data  # Update the data with embeddings

# --- Model Definition (Using Gemini Embeddings and LSTM) ---
class RULPredictionModel(nn.Module):
    def __init__(self, embedding_dim, hidden_size, num_layers=2, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)  # Output layer for RUL prediction

    def forward(self, x):
        # Pass embeddings through LSTM
        out, _ = self.lstm(x)
        # Take the output from the last time step
        out = out[:, -1, :]
        # Pass through fully connected layer for RUL prediction
        out = self.fc(out)
        return out

# --- Training Function ---
def train_model(model_name, train_dataset_path, eval_dataset_path,
                staging_bucket, bucket_name, base_output_dir,
                project_id, region,  # Add project_id and region arguments
                use_rolling_features=False, tuned_model_display_name_or_index=None):  # Add tuned_model_display_name

    global logging

    logging.info("Training configuration:")
    logging.info(f"Model name: {model_name}")
    logging.info(f"Train Dataset Path: {train_dataset_path}")
    logging.info(f"Eval Dataset Path: {eval_dataset_path}")
    logging.info(f"Staging Bucket: {staging_bucket}")
    logging.info(f"Bucket Name from args: {bucket_name}")  # Log bucket_name
    logging.info(f"Base Output Dir: {base_output_dir}")
    logging.info(f"Project ID: {project_id}")  # Log project_id
    logging.info(f"Region: {region}")  # Log region
    logging.info(f"Use Rolling Features: {use_rolling_features}")

    # Initialize Vertex AI with project_id and region
    vertexai.init(project=project_id, location=region)

    # 1. Model Initialization (Modified to handle both cases)
    vertexai_index=10
    models = aiplatform.Model.list()
    name=models[vertexai_index].display_name
    resource_name=models[vertexai_index].resource_name
    #print(f"name: {name}")
    #print(f"resource_name: {resource_name}")

    try:
        # Try loading by display name first
        models = aiplatform.Model.list()
        if models:
            base_model = models[vertexai_index]
            logging.info(f"Loaded {base_model.resource_name} by display name.")
        else:
            # If not found, assume it's an index
            vertexai_index = int(tuned_model_display_name_or_index)
            models = aiplatform.Model.list()
            if 0 <= vertexai_index < len(models):
                base_model = models[vertexai_index]
                logging.info(f"Loaded {base_model.resource_name} by index.")
            else:
                raise ValueError(f"Invalid vertexai_index or display_name: {tuned_model_display_name_or_index}")
    except ValueError as e:
        logging.error(f"Error loading model: {e}")
        raise
    except Exception as e:
        logging.error(f"Unexpected error loading model: {e}")
        raise

    # 2. Fine-tuning Job
    try:
        # fine_tuning_job = jobs.FineTuningJob(
        #     model=base_model,
        #     training_data_uri=train_dataset_path,
        #     validation_data_uri=eval_dataset_path,
        # )

        # fine_tuned_model = fine_tuning_job.run(timeout=3600)
        fine_tuned_model = base_model
    except Exception as e:
        logging.error(f"Error in fine-tuning job: {e}")
        raise e

    # 3. Create Datasets

    train_dataset = CMAPSSJSONLDataset(train_dataset_path, use_gcs=True, bucket_name=bucket_name)  # Pass bucket_name
    eval_dataset = CMAPSSJSONLDataset(eval_dataset_path, use_gcs=True, bucket_name=bucket_name)  # Pass bucket_name

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    eval_loader = DataLoader(eval_dataset, batch_size=64)

    # 4. Model and Optimizer Setup
    embedding_dim = fine_tuned_model.predict(instances=[{"content": "test"}]).predictions[0].shape[0]
    hidden_size = 128
    model = RULPredictionModel(embedding_dim, hidden_size)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    # 5. Training Loop
    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for embeddings, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(embeddings)
            targets = targets.unsqueeze(1).float()
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * embeddings.size(0)

        train_loss /= len(train_loader.dataset)

        # 6. Evaluation
        model.eval()
        eval_loss = 0.0
        with torch.no_grad():
            for embeddings, targets in eval_loader:
                outputs = model(embeddings)
                targets = targets.unsqueeze(1).float()
                loss = criterion(outputs, targets)
                eval_loss += loss.item() * embeddings.size(0)

        eval_loss /= len(eval_loader.dataset)

        logging.info(f"Epoch [{epoch + 1}/{num_epochs}], "
                     f"Train Loss: {train_loss:.4f}, "
                     f"Eval Loss: {eval_loss:.4f}")


    # 7. Save the Model to GCS Bucket
    model_dir = os.path.join(base_output_dir, model_name)
    model_path = os.path.join(model_dir, 'model-gemini-nasa-cmapss.pth')

    # Save the model locally first
    torch.save(model.state_dict(), "model-gemini-nasa-cmapss.pth")

    # Upload the model to GCS
    try:
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(model_path)  # Use the full GCS path for the blob
        blob.upload_from_filename("model-gemini-nasa-cmapss.pth")  # Upload the local file

        logging.info(f"Model uploaded to: gs://{bucket_name}/{model_path}")
    except Exception as e:
        logging.error(f"Error uploading model to GCS: {e}")
        raise


# --- Main Execution ---
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--model', type=str, required=True, help='Name of the model.')
    parser.add_argument('--train_dataset', type=str, required=True, help='Path to the training dataset.')
    parser.add_argument('--eval_dataset', type=str, required=True, help='Path to the evaluation dataset.')
    parser.add_argument('--staging_bucket', type=str, required=True, help='GCS staging bucket.')
    parser.add_argument('--bucket_name', type=str, required=True, help='GCS bucket name.')
    parser.add_argument('--base_output_dir', type=str, required=True, help='Base output directory.')
    parser.add_argument('--project_id', type=str, required=True, help='Google Cloud Project ID.')  # Add project_id argument
    parser.add_argument('--region', type=str, required=True, help='Google Cloud Region.')  # Add region argument
    parser.add_argument('--use_rolling_features', action='store_true',
                        help='Whether to use rolling features for training.')
    parser.add_argument('--tuned_model_display_name_or_index', type=str, required=True,
                        help='Display name or index of the tuned Gemini model to use.')
    args = parser.parse_args()

    train_model(args.model, args.train_dataset, args.eval_dataset,
                args.staging_bucket, args.bucket_name, args.base_output_dir,
                args.project_id, args.region,
                args.use_rolling_features, args.tuned_model_display_name_or_index)
"""

In [ ]:
import os
from google.cloud import aiplatform
import argparse
import json
import logging
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import time
import subprocess
from google.cloud import storage
import pandas as pd
from typing import List
import numpy as np
import torch.utils.data as data
import vertexai
from vertexai.preview.language_models import TextGenerationModel


# --- Main Script ---
if __name__ == "__main__":
    # Get project details from environment variables
    PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
    REGION = os.environ.get("GOOGLE_CLOUD_REGION")
    SERVICEACCOUNT = os.environ.get("GOOGLE_CLOUD_SERVICEACCOUNT")
    PROJECT_NUMBER = os.environ.get("GOOGLE_CLOUD_PROJECT_NUMBER")
    BUCKET_NAME = os.environ.get("GOOGLE_CLOUD_BUCKET_NAME")
    STAGING_BUCKET = f"gs://{BUCKET_NAME}/staging"

    # Initialize Vertex AI
    aiplatform.init(project=PROJECT_ID, location=REGION)

    # --- Data and Output Paths ---
    TRAINING_DATA_PATH = f"gs://{BUCKET_NAME}/cmapss_FD004_train_text.jsonl"
    EVAL_DATA_PATH = f"gs://{BUCKET_NAME}/cmapss_FD004_test_text.jsonl"
    BASE_OUTPUT_DIR = f"gs://{BUCKET_NAME}/model_output"

    # Create or overwrite trainer/train.py
    os.makedirs('trainer', exist_ok=True)
    with open('trainer/train.py', 'w') as f:
        f.write(train_py_content)

    # Create and run the custom training job
    job = aiplatform.CustomTrainingJob(
    display_name="cmapss-rul-gemini-finetuning",
    script_path="trainer/train.py",
    requirements=["google-cloud-aiplatform", "google-generativeai", "transformers", "pandas", "torch", "tensorflow"], # Added tensorflow to the requirements
    container_uri="us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.2-4.py310:latest",  # Updated container URI
    staging_bucket=STAGING_BUCKET,
    model_serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/pytorch-gpu.2-4:latest"  # Updated serving container URI
    )


    model = job.run(
        replica_count=1,
        machine_type="a2-highgpu-1g",  # Machine type with A100 GPU
        accelerator_type="NVIDIA_TESLA_A100",  # A100 accelerator
        accelerator_count=1,
        base_output_dir=BASE_OUTPUT_DIR,
        args=['--model', 'rul_predictor_cmapss',
              '--train_dataset', TRAINING_DATA_PATH,
              '--eval_dataset', EVAL_DATA_PATH,
              '--staging_bucket', STAGING_BUCKET,
              '--bucket_name', BUCKET_NAME,
              '--base_output_dir', BASE_OUTPUT_DIR,
              '--project_id', PROJECT_ID,  # Add PROJECT_ID argument
              '--region', REGION],  # Add REGION argument
        # Add timeout to prevent the job from running indefinitely if it's stuck
        timeout=3600,  # Timeout in seconds (1 hour)
        sync=True # Added this line
    )

    model.wait()  # Wait for training to complete

    print('\n\n')
    print(f"Model name: {model.display_name}")
    print(f"Model resource name: {model.resource_name}")
    print(f"Model training start time: {model.create_time}")
    print(f"Model training end time: {model.update_time}")
    print(f"Model training duration: {model.update_time - model.create_time}")
    print('\n')

    # Accessing metrics from training job completion statistics metadata
    if model.training_job and hasattr(model.training_job, 'completion_stats') and model.training_job.completion_stats and model.training_job.completion_stats.metadata:
        metrics = model.training_job.completion_stats.metadata
        # Use .get() for safety
        print(f"Model training loss: {metrics.get('train_loss')}")
        print(f"Model evaluation loss: {metrics.get('eval_loss')}")
        print(f"Model evaluation metrics: {metrics}")
    else:
        print("Training job or completion statistics metadata not available.")

    print('\n')

    logging.info(f"Fine-tuned model: {model.resource_name}")

## MODEL EVALUATION

In [ ]:
from google.cloud import storage
import torch

# Initialize the GCS client
client = storage.Client()

# Specify the bucket and blob (file)
bucket_name = os.environ.get("GOOGLE_CLOUD_BUCKET_NAME")
bucket = client.bucket(bucket_name)
blob_name = "model_output/model-nasa-gpu.pth"

blob_name = "model_output/model-nasa-gpu.pth"
bucket = client.bucket(bucket_name)
blob = bucket.blob(blob_name)

# Download the file to a local path
local_file_path = "model-nasa-gpu.pth"  # Or your desired local path
blob.download_to_filename(local_file_path)

print(f"Downloaded '{blob_name}' from '{bucket_name}' to '{local_file_path}'")

Downloaded 'model_output/model-nasa-gpu.pth' from 'poc-my-new-staging-bucket-2025-1' to 'model-nasa-gpu.pth'


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Assuming this is how the CMAPSSJSONLDataset is defined in the document
import torch
from torch.utils import data
import json
from google.cloud import storage
import logging
import os

class CMAPSSJSONLDataset(data.Dataset):
    def __init__(self, data_path, sequence_length=30):
        self.data = []
        self.sequence_length = sequence_length

        # Check if data_path is a local file or a GCS URI
        if data_path.startswith('gs://'):
            # If GCS URI, download to a temporary file
            storage_client = storage.Client()
            bucket_name = data_path.split('/')[2]
            blob_name = '/'.join(data_path.split('/')[3:])
            bucket = storage_client.bucket(bucket_name)
            blob = bucket.blob(blob_name)
            tmp_file = "/tmp/temp_data.jsonl"
            blob.download_to_filename(tmp_file)
            data_file = tmp_file
        else:
            # If local file, use it directly
            data_file = data_path

        with open(data_file, 'r') as f:
            for line in f:
                try:
                    record = json.loads(line)
                    sequence = torch.tensor(record["sequence"], dtype=torch.float32)
                    rul = torch.tensor([record["rul"]], dtype=torch.float32)
                    self.data.append((sequence, rul))
                except json.JSONDecodeError as e:
                    logging.warning("Skipping invalid JSON line: %s, Error: %s", line, e)

        # Remove the temporary file if it was created
        if data_path.startswith('gs://'):
            os.remove(tmp_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

class RULPredictionModel(nn.Module):
    #LSTM-based model for RUL prediction
    def __init__(self, input_size, hidden_size, num_layers=3, dropout=0.3): # Match the architecture in train.py
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_size // 2, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc1(out[:, -1, :])
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out

import numpy as np

def calculate_cmapss_score(true_rul, predicted_rul):
    """Calculates a simplified CMAPSS score."""
    d = np.array(predicted_rul) - np.array(true_rul)  # Difference between predicted and true RUL
    score = sum([
        np.exp(-d[i] / 13) - 1 if d[i] < 0 else np.exp(d[i] / 10) - 1
        for i in range(len(d))
    ])
    return score


def evaluate_model(model_path, eval_dataset_path, input_size, hidden_size, sequence_length):
    """Evaluates the trained RUL prediction model."""

    # Load the saved model using the correct model architecture
    model = RULPredictionModel(input_size, hidden_size, num_layers=3)  # Change num_layers to 3 to match the training architecture

    # Explicitly load the model to the CPU
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda')))

    model.eval()  # Set to evaluation mode



    # Load the evaluation dataset
    eval_dataset = CMAPSSJSONLDataset(eval_dataset_path, sequence_length)
    eval_loader = DataLoader(eval_dataset, batch_size=64, shuffle=False)  # No need to shuffle for evaluation

    # Make predictions and calculate metrics
    all_predictions = []
    all_targets = []
    with torch.no_grad():  # Disable gradient calculations during evaluation
        for sequences, ruls in eval_loader:
            predictions = model(sequences)
            all_predictions.extend(predictions.flatten().tolist())
            all_targets.extend(ruls.flatten().tolist())

    # Calculate evaluation metrics (e.g., MAE, RMSE)
    mae = mean_absolute_error(all_targets, all_predictions)
    rmse = np.sqrt(mean_squared_error(all_targets, all_predictions))
    mse = mean_squared_error(all_targets, all_predictions)  # Calculate MSE

    # Calculate CMAPSS score
    cmapss_score = calculate_cmapss_score(all_targets, all_predictions)

    # Print the results
    print(f"Evaluation Results:")
    print(f"Average Evaluation Loss (MSE): {mse:.2f}")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
    print(f"CMAPSS Score: {cmapss_score:.2f}")



# --- Example Usage (if __name__ == "__main__": block) ---
if __name__ == "__main__":
    model_path = "model-nasa-gpu.pth"  # Replace with the actual path to your saved model
    eval_dataset_path = f"gs://{BUCKET_NAME}/cmapss_FD004_test_sequences.jsonl"  # Replace with your evaluation data path (GCS URI)
    input_size = 25  # Updated to match the input size used during training
    hidden_size = 128 # Updated to match the hidden size used during training
    sequence_length = 30  # Replace with the sequence length used during training

    evaluate_model(model_path, eval_dataset_path, input_size, hidden_size, sequence_length)

Evaluation Results:
Average Evaluation Loss (MSE): 675.03
Mean Absolute Error (MAE): 25.98
Root Mean Squared Error (RMSE): 25.98
CMAPSS Score: 1607.38


N=10

Evaluation Results:
Average Evaluation Loss (MSE): 675.91
Mean Absolute Error (MAE): 26.00
Root Mean Squared Error (RMSE): 26.00
CMAPSS Score: 1609.80

N=200

Evaluation Results:
Average Evaluation Loss (MSE): 676.00
Mean Absolute Error (MAE): 26.00
Root Mean Squared Error (RMSE): 26.00
CMAPSS Score: 1610.04



In [ ]:
import os

print("GOOGLE_CLOUD_PROJECT:", os.environ.get("GOOGLE_CLOUD_PROJECT"))
print("GOOGLE_CLOUD_REGION:", os.environ.get("GOOGLE_CLOUD_REGION"))
print("GOOGLE_CLOUD_SERVICEACCOUNT:", os.environ.get("GOOGLE_CLOUD_SERVICEACCOUNT"))
print("GOOGLE_CLOUD_PROJECT_NUMBER:", os.environ.get("GOOGLE_CLOUD_PROJECT_NUMBER"))
print("GOOGLE_CLOUD_BUCKET_NAME:", os.environ.get("GOOGLE_CLOUD_BUCKET_NAME"))

In [ ]:
VERTEX_AI_SERVICE_AGENT = !gcloud projects describe $PROJECT_ID --format='value(project_number)' | xargs -I{} gcloud iam service-accounts list --filter="displayName:Compute Engine default service account" --project=$PROJECT_ID --format="value(email)"
VERTEX_AI_SERVICE_AGENT = VERTEX_AI_SERVICE_AGENT[0].strip()  # Access the first element (index 0)
print("Vertex AI Service Agent:", VERTEX_AI_SERVICE_AGENT)